In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from PIL import Image
from joblib import dump, load

%matplotlib inline

In [261]:
# Importing training data.
data_train = pd.read_csv('mnist_train.csv')
len(data_train)

60000

In [272]:
# Splitting into features and labels.
x_train = data_train.drop('label', axis=1).astype('float32')/255
y_train = data_train['label']
len(x_train), len(y_train)

(60000, 60000)

In [264]:
# Finding out the unique labels.
unique_labels = np.unique(y_train)
unique_labels

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64)

In [265]:
def encode_labels(y):
    """
    This function converts the labels into their
    one-hot encoded equivalent.
    """
    y_enc = []
    for i in range(len(y)):
        y_enc.append((y[i] == unique_labels).astype(int))
        
    return np.array(y_enc)

In [266]:
# Encoding the training labels.
y_train_encoded = encode_labels(y_train)
y_train_encoded

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]])

In [267]:
# Instantiate model.
model = MLPClassifier(verbose=True,
                      random_state=10,
                      shuffle=True,
                      early_stopping=True,
                      max_iter=500)

# Fit data to the model.
model.fit(x_train, y_train_encoded)

Iteration 1, loss = 1.34580704
Validation score: 0.805833
Iteration 2, loss = 0.62875170
Validation score: 0.857667
Iteration 3, loss = 0.48572432
Validation score: 0.885833
Iteration 4, loss = 0.39265251
Validation score: 0.903833
Iteration 5, loss = 0.32961198
Validation score: 0.912167
Iteration 6, loss = 0.28377151
Validation score: 0.924000
Iteration 7, loss = 0.25076370
Validation score: 0.926500
Iteration 8, loss = 0.22515969
Validation score: 0.933167
Iteration 9, loss = 0.20533962
Validation score: 0.932167
Iteration 10, loss = 0.18786829
Validation score: 0.938333
Iteration 11, loss = 0.17304292
Validation score: 0.936333
Iteration 12, loss = 0.15933087
Validation score: 0.939667
Iteration 13, loss = 0.14766584
Validation score: 0.940167
Iteration 14, loss = 0.13606382
Validation score: 0.939000
Iteration 15, loss = 0.12645194
Validation score: 0.943000
Iteration 16, loss = 0.11772558
Validation score: 0.944500
Iteration 17, loss = 0.10935074
Validation score: 0.942500
Iterat

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=True, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=10, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=True,
              warm_start=False)

In [268]:
# Saving trained model.
dump(model, 'digit_classifier_t1.joblib')

['digit_classifier_t1.joblib']

In [8]:
# Loading saved model.
model = load('digit_classifier_t1.joblib')

In [270]:
# Importing test data.
data_test = pd.read_csv('mnist_test.csv')
len(data_test)

10000

In [271]:
# Splitting into features and labels.
x_test = data_test.drop('label', axis=1).astype('float32')/255
y_test = data_test['label']
len(x_test), len(y_test)

(10000, 10000)

In [273]:
# Encoding the test labels.
y_test_encoded = encode_labels(y_test)
y_test_encoded

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [274]:
# Scoring the model.
model.score(x_test, y_test_encoded)

0.9584

In [4]:
def process_pred(pred):
    """
    This function returnns the digit predicted
    by the model.
    """
    return np.argmax(pred)

In [5]:
def resize_center_image(old_im):
    old_im = old_im.resize((20,20))
    old_size = old_im.size

    new_size = (28, 28)
    new_im = Image.new("L", new_size) 
    new_im.paste(old_im, (4,4))

    return new_im

In [6]:
def process_image(image_path):
    """
    Processes the given image.
    """
    im = Image.open(image_path)
    im = resize_center_image(im)
    return pd.DataFrame(np.asarray(im).flatten().astype(float)/255).T

In [7]:
def show_image(image_path):
    """
    Displays processed image.
    """
    im = Image.open(image_path)
    im = resize_center_image(im)
    return im

In [1]:
IMG_PATH = 'test_img/test_6.png'

In [9]:
show_image(IMG_PATH)

In [10]:
process_image(IMG_PATH)

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
pred = model.predict(process_image(IMG_PATH))

In [12]:
process_pred(pred)

6